# Qualitatitive topic evaluations
This notebook is used for formatting the data for qualitative evaluations. This includes: 
- Inspecting the topic words
- Inspecting representative documents
- Coming up with good "titles" for the topics

In [21]:
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint

In [7]:
def read_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

In [11]:
DATA_DIR = Path("../../BscThesisData/data")
topic_dict = read_pickle(DATA_DIR / "topic_dict.pkl")
doc_topics = pd.read_csv(DATA_DIR / "doc_topics.csv")

doc_topics.dropna(inplace=True)


In [19]:
for topic_key in topic_dict.keys():
    example_docs = doc_topics.loc[doc_topics["topic"] == topic_key, "doc"].sample(5)
    print(f"Results for topic nr. {topic_key}:")
    print(f"word list: {topic_dict[topic_key]}")
    print("representative documents:")
    print(print(example_docs.tolist()))
    print("\n")

Results for topic nr. 0:
word list: ['skatterådet', 'spørger', 'bekræfte', 'bekræfter', 'danmark', 'anses', 'spørgers', 'virksomhed', 'stk', 'omfattet', 'bekræftede', 'salg', 'moms', 'danske', 'selskab', 'fast', 'forbindelse', 'aps', 'skat', 'såfremt', 'finder', 'tale', 'ejendom', 'jf', 'skattepligtig', 'endvidere', 'beskattes', 'momslovens', 'del', 'dansk']
representative documents:
['Skatterådet bekræfter, at Selskab C kan "slankes" pr. 30. juni 2010, således at overført resultat før stiftelse af holdingselskabet den 17. december 2007 bliver udbetalt som skattefri udbytte til de 2 holdingselskaber ejet af Person A og Person B. Endvidere bekræftes det, at E ApS kan sælge sin andel af aktierne i Selskab C A/S, 50 % til F ApS skattefrit.', 'Gærede alkoholholdige produkter der som følge af en efterfølgende rensning og filtrering, fremstår som en farveløs og klar væske, der lugter af alkohol, skal\ntariferes under KN pos. 2208, uanset hvor produktet oprindeligt var tariferet i KN. Såfremt

### Something's rotten in the state of topic 1
It seems to be empty

In [32]:
topic1 = doc_topics[doc_topics["topic"] == 1]
topic1["doc"] = topic1["doc"].str.strip().replace("", np.nan)


C:\Users\jhr\AppData\Local\Temp/ipykernel_12376/2005448945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic1["doc"] = topic1["doc"].str.strip().replace("", np.nan)


,topic,prob,doc
